In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp drive/MyDrive/Colab_Notebooks/utils.py .

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from utils import *
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from zipfile import ZipFile
MIN_IN_DAY = 1440
CUTOFF = 172800

myzip=ZipFile('/content/drive/MyDrive/Colab_Notebooks/load/data.zip')
f=myzip.open('data.csv')
df=pd.read_csv(f)
f.close()
myzip.close()

daily = np.array(df.iloc[:, 1:])[:CUTOFF].T  # 346 residents, 172800 min
daily = daily.reshape((daily.shape[0], -1, MIN_IN_DAY))  # 346 residents, 120 days, 1440 minutes
daily = np.nanmean(daily, axis=1)  # 346 residents, 1440 minutes

normalized_daily = normalized(daily)

In [ ]:
AE_CNN??

In [ ]:
# anomaly dedection, list records by reconstruction error from high to low
latent_dim = 16

cfg = {
    'latent_dim': latent_dim,
    'activation': nn.ReLU(),
}

model_dict = '/content/drive/MyDrive/Colab_Notebooks/load/models/AE_CNN_sigmoid_d={}.pt'.format(latent_dim)
model = AE_CNN(cfg,load_dict=model_dict,device=device)

data = DS(normalized_daily, sep=0, train=False, device=device).data
latent, recon = model(data)
latent = latent.detach().cpu().numpy()

In [ ]:
reconstruction_error = ((recon.squeeze().detach().cpu().numpy() - normalized_daily)**2).sum(1)
order = np.argsort(reconstruction_error)
plt.plot(normalized_daily[order][-50].T)
plt.plot(recon.squeeze().detach().cpu().numpy()[order][-50].T)
plt.title('reconstruction error :'+str(reconstruction_error[order][-50]))

In [ ]:
plt.plot(normalized_daily[order][-50].reshape(72,-1).sum(1))

In [ ]:
plt.plot(reconstruction_error[order],'o')
plt.ylabel('reconstruction error')